<a href="https://colab.research.google.com/github/square-1111/Document-Classification/blob/master/Modeling_and_Training_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Corpus


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
names = ['x', 'y', 'tx', 'ty', 'adj']

In [0]:
import pickle as pkl
import sys
import networkx as nx

In [0]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import numpy as np

In [0]:
from scipy.sparse import csr_matrix

In [0]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [0]:
objects = []
for i in range(len(names)):
    with open("drive/My Drive/GCN/Data/ind.20NG.{}".format(names[i]), 'rb') as f:
      if sys.version_info > (3, 0):
        objects.append(pkl.load(f, encoding='latin1'))
      else:
        objects.append(pkl.load(f))

In [0]:
train_x, train_lab, test_x, test_lab, graph = tuple(objects)

In [0]:
adj = sparse_mx_to_torch_sparse_tensor(graph)

In [0]:
features = torch.FloatTensor(train_x.todense())

In [0]:
test_x = sparse_mx_to_torch_sparse_tensor(test_x)

In [0]:
train_lab = torch.FloatTensor(train_lab)
test_lab = torch.FloatTensor(test_lab)

## Model


In [0]:
class GraphConvolution(Module):
  def __init__(self, in_features, out_features):
    super(GraphConvolution, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    self.bias = Parameter(torch.FloatTensor(out_features))
    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1. / math.sqrt(self.weight.size(1))
    self.weight.data.uniform_(-stdv, stdv)
    self.bias.data.uniform_(-stdv, stdv)

  def forward(self, input, adj):
    support = torch.mm(input, self.weight)
    output = torch.sparse.mm(adj, support)
    return output + self.bias

  def __repr__(self):
    return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'

In [0]:
class GCN(Module):
  def __init__(self, nfeat, nhid, nclass, dropout):
    super(GCN, self).__init__()
    self.gc1 = GraphConvolution(nfeat, nhid)
    self.gc2 = GraphConvolution(nhid, nclass)
    self.dropout = dropout

  def forward(self, x, adj):
    x = F.relu(self.gc1(x, adj))
    x = F.dropout(x, self.dropout, training=self.training)
    x = self.gc2(x, adj)
    return F.log_softmax(x, dim=1)

In [0]:
model = GCN(features.shape[1], 64, 20, 0.5)

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)

In [0]:
if torch.cuda.is_available():
  model = model.cuda()
  features = features.cuda()
  adj = adj.cuda()
  features = features.cuda()
  train_lab = train_lab.cuda()
  # test_x = test_x.cuda()

In [0]:
epoch = 500
idx_train = range(200)

In [0]:
for i in range(epoch):
  model.train()
  optimizer.zero_grad()
  output = model(features, adj)
  loss_train = F.nll_loss(output[idx_train], labels[idx_train])
  acc_train = accuracy(output[idx_train], labels[idx_train])
  loss_train.backward()
  optimizer.step()
  